## Step 21: 연산자 오버로드 (2)

앞서 우리는 연산자 오버로드를 통해 a + b와 같은 코드를 사용할 수 있게 되었다. 하지만 여전히 a * np.array(2.0)처럼 변수와 ndarray 인스턴스를 연산하거나 3 + b처럼 수치 데이터를 함께 사용하지 못한다. 따라서 이번 단계에서는 이를 가능케 해보도록 하겠다.

### 21.1 ndarray와 함께 사용하기

간단하게, a * np.array(2.0)과 같은 코드에서 a가 Variable 인스턴스라면 뒤의 ndarray 인스턴스를 자동으로 Variable 인스턴스로 변환하는 방식으로 구현하겠다.

이를 위해 as_variable이라는 helper function을 만들도록 하겠다. 이 함수는 인수로 주어진 객체를 Variable 인스턴스로 변환해주는 함수이다.

```python
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)
```

이 함수는 인수 obj가 Variable 인스턴스 또는 ndarray 인스턴스라고 가정한다. 두 경우 모두 Variable 인스턴스로 반환하는 것이 이 함수가 하는 역할이다.

이제 이 함수를 Function 클래스의 \_\_call__ 메서드 부분에 추가해주도록 하자.

```python
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]
        
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        ...
```

이제 수정된 클래스를 바탕으로 다음 계산을 수행해보자.

```python
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)
```

\> variable(5.0)

### 21.2 float, int와 함께 사용하기

이어서 float, int, np.float64, np.int64와 같은 수치 데이터를 함께 사용할 수 있도록 해보겠다.

```python
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)
```

위와 같이 add 메서드에서 수치 데이터 인수를 array로 바꿔주는 부분을 추가해주면 간단하게 처리 가능하다.

```python
x = Variable(np.array(2.0))
y = x + 3.0

print(y)
```

\> variable(5.0)

이로써 수치 데이터와 Variable 인스턴스를 조합한 계산이 가능해졌다. 여기서는 add 함수만 수정했지만 mul 등 다른 함수들도 같은 방식으로 수정 가능하다.

다만 이러한 방식은 두 가지 문제를 가지고 있다.

### 21.3 문제점1: 첫 번째 인수가 float이나 int인 경우

현재의 DeZero는 x * 2.0을 정상적으로 계산할 수 있다. 하지만 2.0 * x와 같은 계산을 수행할 경우 에러가 발생한다.

> TypeError: unsupported operand type(s) for *: 'float' and 'Variable'

이는 우리가 add 함수를 수정할 때 두 번째 인수가 수치 데이터이리라고 가정하고 수정했기 때문이다.

2.0 * x, 즉 수치 데이터가 앞쪽 인수인 코드의 경우 다음과 같은 순서로 계산이 진행되고 오류가 발생한다.

1. 첫 번째 인수(연산자 왼쪽)인 2.0의 \_\_mul__ 메서드를 호출하려 시도
2. 그러나 float 타입에 대한 \_\_mul__ 메서드가 구현되어 있지 않음
3. 이어서 두 번째 인수(연산자 오른쪽)인 x의 특수 메서드 \_\_rmul__ 메서드 호출 시도
4. 그러나 Variable 인스턴스에는 \_\_rmul__ 메서드가 구현되어 있지 않음

여기서 핵심은 *와 같은 이항 연산자의 경우 피연산자(항)의 위치에 따라 호출되는 특수 메서드가 달라진다는 점이다. 곰셉의 경우 피연산자가 좌항이면 \_\_mul__ 메서드가, 우항이면 \_\_rmul__ 메서드가 호출된다. 따라서 이 문제는 \_\_rmul__의 구현으로 해결할 수 있다.

```python
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul
```

```python
x = Variable(np.array(2.0))
y = 3.0 * x + 1.0
print(y)
```

\> variable(7.0)

이제 Variable 인스턴스와 float, int를 함께 사용할 수 있다. 이어서 나머지 문제 하나를 마저 해결해보자.

### 21.4 문제점 2: 좌항이 ndarray 인스턴스인 경우

남은 문제는 ndarray 인스턴스가 좌항이고 Variable 인스턴스가 우항인 경우이다. 이 문제를 해결하기 위해서는 '연산자 우선순위'를 지정해야 한다.

구체적으로는 Variable 인스턴스 속성에 \_\_array_priority__를 추가하고 그 값을 큰 정수로 설정해야 한다.

```python
class Variable:
    __array_priority__ = 200
    ...
```

이렇게 하면 Variable 인스턴스의 연산자 우선순위를 ndarray 인스턴스의 연산자 우선순위보다 높일 수 있다. 이 결과 좌항이 ndarray 인스턴스라 해도 우항인 Variable 인스턴스의 연산자 메서드가 우선 호출된다.